# Loading The Required Librariries

Using these libraries as:

- tweepy for reading tweets
- pandas for stroring data
- jsonlines for jsonl parsing

In [ ]:
import tweepy
import pandas as pd
import jsonlines

# Initialising The Credentials

Defining the credentials used for extracting the tweets

In [ ]:
Consumer_Key = "" 
Consumer_Key_Secret = ""
Access_Key = ""
Access_Key_Secret = ""

# Function to Extract Tweets

In [ ]:
def get_tweets(username): 
    
    
        #Authentication Using the Credentials 
        auth = tweepy.OAuthHandler(Consumer_Key, Consumer_Key_Secret) 
        auth.set_access_token(Access_Key, Access_Key_Secret) 
        api = tweepy.API(auth) 
        
        #Selecting the 200 tweets (Limit)
        tweets = api.user_timeline(username,count=200)
        
        #Initialising the Empty List 
        twts=[]  
        
        #Appending all the tweets in the twts list
        for j in tweets: 
            twts.append(j._json)  
            
        #Dumping the data into the JSONL file
        with jsonlines.open('MIDAS_Tweets.jsonl', 'w') as twt:
            twt.write(twts)

# Parsing the Data

In [ ]:
def parse_tweets(path):
    
    #Initialising the list
    twts = None
    
    #Copying the data from jsonl to the list
    with jsonlines.open(path, 'r') as t:
        twts = t.read()
        
    #Creating the dataframe's structure
    Data = [[twt['text'], twt['created_at'], twt['favorite_count'], twt['retweet_count']] for twt in twts]
    Col = ['Text', 'Created At', 'Number of Likes', 'Number of Retweets']
    
    #Creating the Dataframe
    datfram = pd.DataFrame(data = Data, columns = Col)
    
    #Checking for the Occurences of Images
    datfram['Number of Images'] = [len([img for img in twt['extended_entities']['media'] if img['type'] == 'photo'])
                        if 'extended_entities' in twt else None for twt in twts]
    
    return datfram

# Driver Code

In [ ]:
#Getting the Tweets and storing them in jsonl file
get_tweets("midasIIItd")  

#Geeting the data from jsonl file and storing them in csv file
parse_tweets_from_json('MIDAS_Tweets.jsonl').to_csv('MIDAS_Tweets.csv', header=True, index=False, encoding = 'utf-8')

# Displaying the Data

In [ ]:
midas_df = pd.read_csv('MIDAS_Tweets.csv')
midas_df.head(10)